# Preprocession of DESI spectrum data

This notebook is aimed at preprocessing the DESI spectrum data. There are several changes we make to the original data:

* Rebin the spectrum in the restframe.
* Clip the abnormal points from the spectrum. (3 sigma)
* Normalize.

**This notebook is not completed**

In [114]:
from dla_cnn.desi.DesiMock import DesiMock
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.io import fits
from scipy.interpolate import interp1d
from matplotlib.backends.backend_pdf import PdfPages
from tqdm import tqdm
from astropy.stats import sigma_clip
from scipy.optimize import curve_fit

In [76]:
def get_spilt_point(data:DesiMock, id):
    line_b = data.get_sightline(id=id, camera='b')
    line_z = data.get_sightline(id=id, camera='z')
    line_r = data.get_sightline(id=id, camera='r')
    spilt_loglam_br = np.average([np.max(line_b.loglam), np.min(line_r.loglam)])
    spilt_loglam_rz = np.average([np.min(line_z.loglam), np.max(line_r.loglam)])
    return spilt_loglam_br, spilt_loglam_rz

def get_between(array, max, min, maxif=False, minif=False):
    if maxif:
        if minif:
            return np.intersect1d(np.where(array>=min)[0], np.where(array<=max)[0])
        else:
            return np.intersect1d(np.where(array>min)[0], np.where(array<=max)[0])
    else:
        if minif:
            return np.intersect1d(np.where(array>=min)[0], np.where(array<max)[0])
        else:
            return np.intersect1d(np.where(array>min)[0], np.where(array<max)[0])

def overlap(sightline, data:DesiMock, id):
    '''
    here sightline = data.get_sightline(id=id)
    '''
    spilt_loglam_br, spilt_loglam_rz = get_spilt_point(data, id)
    line_r = data.get_sightline(id=id, camera='r')
    line_b = data.get_sightline(id=id, camera='b')
    line_z = data.get_sightline(id=id, camera='z')
    
    loglam_r = line_r.loglam[0:np.where(loglam == np.max(loglam))[0][0]]
    indice_r = get_between(loglam_r, max=spilt_loglam_rz, min=spilt_loglam_br)
    indice_b = get_between(line_b.loglam, max=spilt_loglam_br, min=0, maxif=True)
    indice_z = get_between(line_z.loglam, max=np.Infinity, min=spilt_loglam_rz, minif=True)
    
    loglam_r, loglam_b, loglam_z = loglam_r[indice_r], line_b.loglam[indice_b], line_z.loglam[indice_z]
    flux_r, flux_b, flux_z = line_r.flux[indice_r], line_b.flux[indice_b], line_z.flux[indice_z]
    sightline.loglam = np.concatenate((loglam_b, loglam_r, loglam_z))
    sightline.flux = np.concatenate((flux_b, flux_r, flux_z))

In [ ]:
def rest_lam(sightline):
    loglam = sightline.loglam
    z = sightline.z_qso
    sightline.loglam_rest = loglam - np.log10(1+z)

In [87]:
def get_intensity(sightline):
    lam = 10**sightline.loglam_rest
    flux = sightline.flux
    delta_lam = lam[1:] - lam[0:-1]
    flux_avg = (flux[1:] + flux[0:-1]) / 2
    sightline.intensity = np.sum(delta_lam * flux_avg) * 1e-17

In [ ]:
def clip(sightline):
    